# Multi-Shot Prompting and In-Context Learning

## Just a quick definition

**Multi-shot** prompting is a technique in which multiple examples are provided within a prompt to guide a language model’s response, improving its ability to generalize and perform tasks based on the demonstrated patterns.

In [1]:
import os
from openai import OpenAI
from IPython.display import display, Markdown
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [2]:
# define response function

client = OpenAI()

def get_response(prompt: str, model: str="gpt-5-chat-latest") -> str:
    response = client.responses.create(
        model=model,
        input=[
            {"role": "user","content": prompt}
        ]
    )
    return response.output_text


### 1. Basic Multi-Shot Learning

In [3]:
PROMPT = """

Classify the following text into one of the following categories:

1. Technology
2. Science
3. Art
4. History

Text: The advancements in quantum computing have the potential to revolutionize the field of cryptography, enabling faster processing and more secure communication methods.

EXAMPLES

Q: The invention of the printing press in the 15th century allowed for the mass production of books and the spread of knowledge.
A: Technology

Q: The discovery of penicillin in 1928 marked the beginning of modern antibiotics and has saved countless lives.
A: Science

Q: The Mona Lisa, painted by Leonardo da Vinci in the 16th century, is one of the most famous works of art in history.
A: Art

Q: The fall of the Berlin Wall in 1989 was a significant event in world history, symbolizing the end of the Cold War.
A: History

"""

# get response
response_text = get_response(PROMPT, model="gpt-5-chat-latest")
display(Markdown(response_text))

A: Technology

### 2. In-Context Learning of Tone and Styles

In [5]:
PROMPT = """
You are a social media copywriter for the fictional company "BrightWave Electronics." Your task is to write social media posts that reflect the company's unique tone and style.

Tone: Friendly, upbeat, and approachable. Posts should inspire excitement about technology and innovation, while remaining accessible to a broad audience.

Style: Use concise sentences, positive language, and occasional emojis to convey enthusiasm. Posts should highlight product benefits, company values, and community engagement. Avoid jargon and keep the message clear and engaging.

EXAMPLES

Q: Post about announcing our new wireless earbuds
A: Announcing our new wireless earbuds! Experience crystal-clear sound and all-day comfort. Ready to upgrade your listening? 🎧✨ #BrightWaveElectronics

Q: Post about our commitment to sustainability
A: We're committed to a sustainable future! 🌍💚 At BrightWave Electronics, we prioritize eco-friendly materials and energy-efficient designs. Join us in making a positive impact! #SustainabilityMatters

Q: Post about our community engagement initiatives
A: At BrightWave Electronics, we believe in giving back! 💙✨ We're proud to support local schools and organizations through technology donations and educational programs. Together, we can make a difference! #CommunityFirst


I want you to generate a social media post for the latest technological research we are conducting.

"""

In [6]:
response_text = get_response(PROMPT, model="gpt-5-chat-latest")
display(Markdown(response_text))

Big things are happening in our labs! 🚀💡 At BrightWave Electronics, our team is exploring the next wave of smart technology to make everyday life easier and more connected. We're turning bold ideas into tomorrow’s innovations—stay tuned for what’s next! 🔬✨ #TechInnovation #BrightWaveElectronics

### 3. Prompt Templates

In [7]:
# Class to define prompt template
class PromptTemplate:
    def __init__(self, template: str, input_variables: list[str]):
        self.template = template
        self.input_variables = input_variables

    def generate(self, **kwargs) -> str:
        return self.template.format(**{k: kwargs[k] for k in self.input_variables})

In [8]:
# Basic prompt template
basic_template = PromptTemplate(
    template="Give me a 1 sentence introduction to the following topic: {topic}",
    input_variables=["topic"]
)

In [9]:
# Advanced prompt template
advanced_template = PromptTemplate(
    template="Given my age: {age}, height: {height}, and weight: {weight}, generate a fitness plan.",
    input_variables=["age", "height", "weight"]
)

In [10]:
basic_template.generate(topic="Quantum Computing")

'Give me a 1 sentence introduction to the following topic: Quantum Computing'

In [11]:
advanced_template.generate(age=30, height=175, weight=70)

'Given my age: 30, height: 175, and weight: 70, generate a fitness plan.'

### 4. Advanced Multi-Shot Learning

In [12]:
# define our template
multi_shot_template = PromptTemplate(
    template="""
    Perform the specified task on the given text.
        
    Examples:
    Text: I love this product! It's amazing.
    Task: sentiment
    Result: Positive
    
    Text: Bonjour, comment allez-vous?
    Task: language
    Result: French
    
    Now, perform the following task:
    Text: {input_text}
    Task: {task}
    Result:

""",
input_variables=["input_text", "task"]
)

In [13]:
PROMPT1 = multi_shot_template.generate(input_text="I don't think I like this product", task="sentiment")
PROMPT2 = multi_shot_template.generate(input_text="Das Wetter ist heute sehr schön und die Sonne scheint.", task="language")

In [14]:
PROMPT1

"\n    Perform the specified task on the given text.\n\n    Examples:\n    Text: I love this product! It's amazing.\n    Task: sentiment\n    Result: Positive\n\n    Text: Bonjour, comment allez-vous?\n    Task: language\n    Result: French\n\n    Now, perform the following task:\n    Text: I don't think I like this product\n    Task: sentiment\n    Result:\n\n"

In [14]:
PROMPT2

"\n    Perform the specified task on the given text.\n\n    Examples:\n    Text: I love this product! It's amazing.\n    Task: sentiment\n    Result: Positive\n\n    Text: Bonjour, comment allez-vous?\n    Task: language\n    Result: French\n\n    Now, perform the following task:\n    Text: Das Wetter ist heute sehr schön und die Sonne scheint.\n    Task: language\n    Result:\n\n"

In [15]:
print(get_response(PROMPT1))

Result: Negative


In [16]:
print(get_response(PROMPT2))

Result: German
